In [ ]:
import os
from typing import TypedDict, Annotated
from dotenv import load_dotenv
from dotenv.variables import Literal
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

In [ ]:
gemini_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
gemini_api_key

In [ ]:
class SummarizerState(TypedDict):
    input: str
    messages: Annotated[list[BaseMessage], add_messages]
    critique_score: int
    summary: str
    feedback: str
    threshold: int
    counter: int

In [ ]:
model1 = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    api_key=gemini_api_key,
)
model2 = ChatOllama(
    model='deepseek-r1:1.5b',
    temperature=0.4
)

In [ ]:
class Summary(BaseModel):
    summary: str = Field(description='Summarized text created by LLM')

In [ ]:
summarizer_input_prompt = ChatPromptTemplate(
    [
        (SystemMessage(content="You are a summarizing agent that creates summaries of provided news snippets or text and keeps them under 500 words or tokens. Use the following feedback given by the critique to improve your summary: {feedback}. If no feedback is given just generate a summary as normal")),
        (HumanMessage(content="{user_input}"))
    ]
)

In [ ]:
structured_summarizer_model = summarizer_input_prompt | model1.with_structured_output(Summary)

In [ ]:
user_input = """
A wiki (/ˈwɪki/ ⓘ WICK-ee) is a form of hypertext publication on the internet which is collaboratively edited and managed by its audience directly through a web browser. A typical wiki contains multiple pages that can either be edited by the public or limited to use within an organization for maintaining its internal knowledge base. Its name derives from the first user-editable website called WikiWikiWeb – wiki (pronounced [wiki][note 1]) is a Hawaiian word meaning 'quick'.[1][2][3][4]

Wikis are powered by wiki software, also known as wiki engines. Being a form of content management system, these differ from other web-based systems such as blog software or static site generators in that the content is created without any defined owner or leader. Wikis have little inherent structure, allowing one to emerge according to the needs of the users.[5] Wiki engines usually allow content to be written using a lightweight markup language and sometimes edited with the help of a rich-text editor.[6] There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Some wiki engines are free and open-source, whereas others are proprietary. Some permit control over different functions (levels of access); for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Further rules may be imposed to organize content. In addition to hosting user-authored content, wikis allow those users to interact, hold discussions, and collaborate.[7]

There are hundreds of thousands of wikis in use, both public and private, including wikis functioning as knowledge management resources, note-taking tools, community websites, and intranets. Ward Cunningham, the developer of the first wiki software, WikiWikiWeb, originally described wiki as "the simplest online database that could possibly work".[8]

The online encyclopedia project Wikipedia is the most popular wiki-based website, as well being one of the internet's most popular websites, having been ranked consistently as such since at least 2007.[9] Wikipedia is not a single wiki but rather a collection of hundreds of wikis, with each one pertaining to a specific language, making it the largest reference work of all time.[10] The English-language Wikipedia has the largest collection of articles, standing at 7,085,230 as of November 2025.[11]"""

In [ ]:
summary = structured_summarizer_model.invoke({
    'user_input': user_input,
})

In [ ]:
class Critique(BaseModel):
    score: int = Field(description='Summary score out of 10')
    feedback: str =  Field(description='feedback for the generated summary to make improvements')

In [ ]:
critique_input_prompt = ChatPromptTemplate(
    [
        (SystemMessage(content='You are a critic agent who is given a summary and the original text and you create a summary score out of 10 based on the following metrics: 1. Conciseness 2. Depth 3.Accuracy 4.Consistency')),
        (AIMessage(content='{summary}'))
    ]
)

In [ ]:
def get_summary(state: SummarizerState):
    user_input = state['input']
    feedback = state['feedback']
    res = structured_summarizer_model.invoke(
        {
            'user_input': user_input,
            'feedback' : feedback,
        }
    )

    return {
        'messages': AIMessage(content=res.summary),
        'summary' : res.summary
    }

def get_critique_score(state: SummarizerState):
    summary = state['summary']

    res = critique_input_prompt.invoke(
        {
            'summary': summary,
        }
    )

    return {
        'messages': AIMessage(content=res.score),
        'critique_score': res.score,
    }

def does_summary_pass(state: SummarizerState)-> Literal['Pass', 'Fail']:

    score = state['critique_score']
    threshold  = state['threshold']

    if score > threshold:
        return 'Pass'
    else:
        return 'Fail'


In [ ]:
graph = StateGraph(SummarizerState)
graph.add_node('summarizer', get_summary)
graph.add_node('critique', get_critique_score)
graph.add_edge('summarizer', 'critique')
graph.add_conditional_edges('critique', does_summary_pass, {

})